## Manual data download

Download files from [JRC-IDEES](https://data.jrc.ec.europa.eu/dataset/jrc-10110-10001) and follow the Excel url. Than, download the country-specific annual demands "JRC-IDEES-2015_All_xlsx_COUNTRY.zip" and pose the sector specific file in your repository or adjust your filename with your download path. 

## Import Python libraries

In [1]:
import pandas as pd
from openpyxl import load_workbook
import os

## Select geographical, sectoral, and temporal scope

In [2]:
all_countries = ['AT', 'BE', 'BG', 'CZ', 'DE', 'DK', 
                 'EE', 'ES', 'FI', 'FR', 'UK', 'HR', 
                 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'EL',
                  'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
selected_countries = all_countries

# GB is named UK in JCR
# GR is named EL in JCR
# missing in JCR: CH, NO
name_clearification = {"UK" : "GB",
                      "EL": "GR"}

In [3]:
sectors = ["Residential", "Tertiary"]
applications = ["water", "space"]

start_year = "2008"
end_year = "2015"

## Preprocessing of JRC-IDEES data

Read the excel country- and sector-specific excel and make a sector-specific dataframe which is placed in the interim folder

In [4]:
def read (sector, application, country_code):
    filename = f"JRC-IDEES-2015_{sector}_{country_code}.xlsx"
    
    sheet_name = "RES_hh_tes" if sector == "Residential" else "SER_hh_tes"
    
    raw = pd.read_excel(filename, header = 0, sheet_name = sheet_name, index_col = 0)
    
    if application == "water":
        row_selection = 'Water heating' if sector == "Residential" else "Hot water"
    else: 
        row_selection = "Space heating"
        
    if country_code == ("UK" or "EL"):
        df = raw.loc[row_selection, start_year:end_year].to_frame().rename(columns = {row_selection: name_clearification[country_code]})
    else:  
        df = raw.loc[row_selection, start_year:end_year].to_frame().rename(columns = {row_selection: country_code})
    df = df.transpose() * 1.163e-2
    return df

In [5]:
interim_path = os.path.realpath('../interim')

In [6]:
for sector in sectors:
    for application in applications:
            pd.concat([
                read(sector, application, country) for country in selected_countries
            ], axis = 0).to_csv(f"{interim_path}/{sector}_{application}.csv", decimal = ",")